# Curve clipping (曲线裁剪)

This method is used to crop curves (该方法用于裁剪曲线)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import math


# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from scipy import interpolate
from pandas import set_option
set_option("display.max_rows", 15)

In [2]:
# 使用GPU训练时候，打开下面注释
import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'

## Set the type of curve to crop (设定要裁剪的曲线种类)

In [3]:
# 定义要处理的常规测井资料
# usual_log_names = ["AC","CNL","DEN","GR","RD","RS"]
# usual_log_names = ["DT","CNL","DEN","GR","RD","RS"]#,"DTXX"]
usual_log_names = ["DTC","DTS"]

# 定义是否要使用常规测井资料，True | False
use_usual_log = True

## Import raw csv data (原始数据导入)

In [4]:
if use_usual_log == True:
    data_path = 'data/usual_logs/'

filename = 'GY1_R_0.1524m.csv'
geo_lable = 'QSK'  # 整段； QSK Layer; QT Layer,
element_logs_file = os.path.join(data_path, filename)
# 读取A、B部分共有数据
un_process = pd.read_csv(element_logs_file,engine='python',encoding='GBK')

## Set the type of curve to crop  (设定裁剪测井曲线长度范围)

In [5]:
# 起始长度和终止长度
# 设定目标范围
start_depth = 5617
end_depth = 6092

In [6]:
un_process

,DEPTH,DTC,DTS
0,5563.8877,-99999.0,-99999.0
1,5564.0401,-99999.0,-99999.0
2,5564.1925,-99999.0,-99999.0
3,5564.3449,-99999.0,-99999.0
4,5564.4973,-99999.0,-99999.0
5,5564.6497,-99999.0,-99999.0
6,5564.8021,-99999.0,-99999.0
...,...,...,...
3511,6098.9641,-99999.0,-99999.0
3512,6099.1165,-99999.0,-99999.0


In [7]:
# 删除NAN行,
# un_process_YS = un_process.dropna(axis=0).drop_duplicates(subset='DEPTH', keep='last')
# 删除重复列
un_process_YS = un_process.dropna(axis=1).drop_duplicates(subset='DEPTH', keep='last')
un_process_YS

,DEPTH,DTC,DTS
0,5563.8877,-99999.0,-99999.0
1,5564.0401,-99999.0,-99999.0
2,5564.1925,-99999.0,-99999.0
3,5564.3449,-99999.0,-99999.0
4,5564.4973,-99999.0,-99999.0
5,5564.6497,-99999.0,-99999.0
6,5564.8021,-99999.0,-99999.0
...,...,...,...
3511,6098.9641,-99999.0,-99999.0
3512,6099.1165,-99999.0,-99999.0


In [8]:
print(un_process_YS.shape)

(3518, 3)


## Determine whether the original data has the target curve (判断原始数据是否具备目标的曲线)

In [9]:
process_YS_cols = (un_process_YS.columns[1:]).tolist()

In [10]:
process_YS_cols

['DTC', 'DTS']

In [11]:

object_cols = []
if use_usual_log == True:
    object_cols = usual_log_names
else:
    object_cols = element_names
print('object_cols:',object_cols)
if set(object_cols).issubset(set(process_YS_cols)):
    if process_YS_cols == object_cols:
        print("Corresponding Exactly!!!")
    else:
        print("corresponding!!!")
else:
    print("logs columns is incompatible!!!")
    missing_logs = [i for i in object_cols if (i not in process_YS_cols)]
    print('missing_logs:',missing_logs)
    exit()

object_cols: ['DTC', 'DTS']
Corresponding Exactly!!!


## Crop(进行裁剪)

In [12]:
C_DEPTH = un_process_YS.loc[:,["DEPTH"]]
C_DEPTH

,DEPTH
0,5563.8877
1,5564.0401
2,5564.1925
3,5564.3449
4,5564.4973
5,5564.6497
6,5564.8021
...,...
3511,6098.9641
3512,6099.1165


In [13]:
un_process_YS.set_index(["DEPTH"], drop=False)
un_process_YS

,DEPTH,DTC,DTS
0,5563.8877,-99999.0,-99999.0
1,5564.0401,-99999.0,-99999.0
2,5564.1925,-99999.0,-99999.0
3,5564.3449,-99999.0,-99999.0
4,5564.4973,-99999.0,-99999.0
5,5564.6497,-99999.0,-99999.0
6,5564.8021,-99999.0,-99999.0
...,...,...,...
3511,6098.9641,-99999.0,-99999.0
3512,6099.1165,-99999.0,-99999.0


In [14]:
# 判断起始长度和终止长度是否在原始测井曲线范围内
# Determine whether the starting and ending lengths are within the range of the original logging curve
if start_depth < min(C_DEPTH.values):
    print("start depth is wrong!")
    exit()
if end_depth > max(C_DEPTH.values):
    print("end depth is wrong!")
    exit()

In [15]:
depth_range = []
# numpy.where() 函数返回输入数组中满足给定条件的元素的索引。
a = np.where(C_DEPTH.DEPTH >= start_depth)[0][0]
b = np.where(C_DEPTH.DEPTH >= end_depth)[0][0]
# if 

start_index = a
end_index =  b

In [16]:
# a = np.where(C_DEPTH.DEPTH <= start_depth)[0][-1]

In [17]:
start_index,end_index

(349, 3466)

In [18]:
C_element = un_process_YS.loc[start_index : end_index]
C_element

,DEPTH,DTC,DTS
349,5617.0753,75.3691,155.1939
350,5617.2277,74.9615,153.6787
351,5617.3801,74.5912,151.0225
352,5617.5325,73.8546,147.7337
353,5617.6849,72.8347,145.1820
354,5617.8373,72.5753,144.4313
355,5617.9897,73.4508,145.5181
...,...,...,...
3460,6091.1917,85.1627,169.4444
3461,6091.3441,85.5282,169.5480


# Saving data to csv(结果写入csv)

In [19]:
import datetime
def tid_maker():
    return '{0:%Y-%m%d-%H%M}'.format(datetime.datetime.now())

In [20]:
csvfile_name = filename.split(".csv")[0] #'TT1-4500m-5070m_R_0.1524.csv'
print(csvfile_name)

高102_R_0.1524m


In [21]:
# C_element.to_csv(data_path + csvfile_name + '_new_'+ str(start_depth) + 'm-'+str(end_depth) + "m_" + tid_maker()+ '.csv',mode='w',index = False,header=True)
C_element.to_csv(data_path + csvfile_name + '_' + geo_lable + '_'+ str(start_depth) + 'm-'+str(end_depth) + "m" + '.csv',mode='w',index = False,header=True)

In [22]:
print("Finished!!")
# print(data_path + csvfile_name + '_new_'+ str(start_depth) + 'm-'+str(end_depth) + "m_" + tid_maker()+ '.csv')
print(data_path + csvfile_name + '_' + geo_lable + '_'+ str(start_depth) + 'm-'+str(end_depth) + "m" + '.csv')

Finished!!
data/usual_logs/高102_R_0.1524m_整段_5617m-6092m.csv


In [23]:
print(tid_maker())

2020-0522-1715
